In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [2]:
########## Dataset ##########

In [3]:
tfds.disable_progress_bar()

In [10]:
dataset = tfds.load('imdb_reviews', as_supervised=True)

In [11]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [13]:
# shuffle  - fills a buffer with `buffer_size` elements, then randomly samples elements
#            from this buffer, replacing the selected elements with new elements.
# batch    - combines consecutive elements of this dataset into batches.
# prefetch - allows later elements to be prepared while the current element is being processed

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [13]:
########## Model ##########

In [16]:
VOCAB_SIZE = 1000
# max_tokens is the maximum size of the vocabulary for this layer.
# The tensors of indices are 0-padded to the longest sequence in the batch (unless you set a fixed output_sequence_length
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [17]:
model = tf.keras.models.Sequential([
    encoder,
    tf.keras.layers.Embedding(input_dim=encoder.vocabulary_size(), output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [18]:
loss = tf.keras.losses.BinaryCrossentropy()

In [19]:
model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

In [22]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10
391/391 [==============================] - 110s 247ms/step - loss: 0.4528 - accuracy: 0.7762 - val_loss: 0.3607 - val_accuracy: 0.8464
Epoch 2/10
391/391 [==============================] - 91s 231ms/step - loss: 0.3401 - accuracy: 0.8566 - val_loss: 0.3298 - val_accuracy: 0.8495
Epoch 3/10
391/391 [==============================] - 91s 232ms/step - loss: 0.3243 - accuracy: 0.8636 - val_loss: 0.3282 - val_accuracy: 0.8615
Epoch 4/10
391/391 [==============================] - 91s 231ms/step - loss: 0.3107 - accuracy: 0.8670 - val_loss: 0.3229 - val_accuracy: 0.8599
Epoch 5/10
391/391 [==============================] - 93s 235ms/step - loss: 0.2903 - accuracy: 0.8777 - val_loss: 0.3353 - val_accuracy: 0.8562
Epoch 6/10
391/391 [==============================] - 91s 231ms/step - loss: 0.2854 - accuracy: 0.8789 - val_loss: 0.3491 - val_accuracy: 0.8568
Epoch 7/10
391/391 [==============================] - 93s 237ms/step - loss: 0.2691 - accuracy: 0.8872 - val_loss: 0.3422 - val_a

In [23]:
model.evaluate(test_dataset, verbose=2)

391/391 - 46s - loss: 0.3812 - accuracy: 0.8524


[0.3812360465526581, 0.8524399995803833]

In [25]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
model.predict(np.array([sample_text]))

array([[0.5769514]], dtype=float32)

In [34]:
model.predict(np.array(['The movie was horrible!']))

array([[0.00467615]], dtype=float32)